# ML Graph Embedding methods

In [5]:
#import libraries
import torch
from torch_geometric.nn import Node2Vec

In [6]:
# Load the graph from the file
server_graph_data_filepath = './data/server_graph_data.pth'
graph_data = torch.load(server_graph_data_filepath)

C:\Users\PC\AppData\Local\Temp\ipykernel_10880\4280630462.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph_data = torch.load(server_graph_data_filepath)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
graph_data = graph_data.to(device)

cuda


In [12]:
# Initialize the Node2Vec model
node2vec = Node2Vec(
    graph_data.edge_index,       # Edge list
    embedding_dim=3,      # Size of embeddings
    walk_length=20,        # Length of each random walk
    context_size=10,       # Window size for Skip-Gram
    walks_per_node=10,     # Number of walks per node
    num_negative_samples=1,  # Number of negative samples for Skip-Gram
    p=0.25,  # Return parameter: encourages staying close to the starting node
    q=4.0,   # In-out parameter: encourages exploring further away
    sparse=True            # Use sparse gradients for efficiency
)

# Define the optimizer
optimizer = torch.optim.SparseAdam(list(node2vec.parameters()), lr=0.01)

# Training loop
def train():
    node2vec.train()
    total_loss = 0
    loader = node2vec.loader(batch_size=128, shuffle=True)
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = node2vec.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Run training for multiple epochs
for epoch in range(1, 101):
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

# Obtain the node embeddings
node_embeddings = node2vec.embedding.weight.data

print("Node Embeddings Shape:", node_embeddings.shape)


Epoch: 010, Loss: 0.8173
Epoch: 020, Loss: 0.7962
Epoch: 030, Loss: 0.7870
Epoch: 040, Loss: 0.7814
Epoch: 050, Loss: 0.7771
Epoch: 060, Loss: 0.7722


KeyboardInterrupt: 

In [11]:
node_embeddings

tensor([[ 0.1956, -0.0389, -0.1859,  ...,  0.0475, -0.0884, -0.0699],
        [-0.0690, -0.0156, -0.1036,  ...,  0.1697,  0.0661, -0.0776],
        [-0.0579, -0.1316, -0.0748,  ..., -0.0214,  0.0254,  0.0597],
        ...,
        [ 0.1938, -0.1235, -0.2061,  ..., -0.1346, -0.0255,  0.0340],
        [-0.2151, -0.1955,  0.2303,  ..., -0.2078, -0.7202,  0.2727],
        [ 0.0382, -0.0080,  0.0493,  ..., -0.0646, -0.0192, -0.0469]])

In [9]:
# Initialize the DeepWalk model by setting p and q to 1 (unbiased random walks)
deepwalk = Node2Vec(
    graph_data.edge_index,
    embedding_dim=64,
    walk_length=40,        # Longer walk length for DeepWalk
    context_size=10,
    walks_per_node=10,
    sparse=True
)

# Define the optimizer
optimizer = torch.optim.SparseAdam(list(deepwalk.parameters()), lr=0.01)

# Training loop
def train():
    deepwalk.train()
    total_loss = 0
    loader = deepwalk.loader(batch_size=128, shuffle=True)
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = deepwalk.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Run training for multiple epochs
for epoch in range(1, 101):
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

# Obtain the node embeddings
node_embeddings_deepwalk = deepwalk.embedding.weight.data

print("Node Embeddings Shape:", node_embeddings.shape)


Epoch: 010, Loss: 0.7447
Epoch: 020, Loss: 0.7291
Epoch: 030, Loss: 0.7280
Epoch: 040, Loss: 0.7271
Epoch: 050, Loss: 0.7272
Epoch: 060, Loss: 0.7271
Epoch: 070, Loss: 0.7266
Epoch: 080, Loss: 0.7260
Epoch: 090, Loss: 0.7252
Epoch: 100, Loss: 0.7248
Node Embeddings Shape: torch.Size([5541, 64])


In [10]:
node_embeddings_deepwalk

tensor([[-0.0688,  0.0457,  0.0361,  ..., -0.1405,  0.0154,  0.0034],
        [ 0.0852,  0.0159,  0.0223,  ...,  0.0041,  0.0731, -0.0527],
        [-0.2004,  0.0200, -0.0227,  ...,  0.1094, -0.0901,  0.0482],
        ...,
        [-0.0477, -0.0120,  0.0903,  ...,  0.1099,  0.0656,  0.0857],
        [-0.1605,  0.3507,  0.3372,  ..., -0.0079,  0.1192, -0.3012],
        [-0.0349, -0.0330,  0.0404,  ..., -0.0570,  0.0777,  0.0656]])